In [1]:
import konlpy
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('./resources/ratings_train.txt', delimiter='\t', keep_default_na=False)

In [3]:
df_train.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
X_train = df_train['document'].values
y_train = df_train['label'].values

In [5]:
df_test = pd.read_csv('./resources/ratings_test.txt', delimiter='\t', keep_default_na=False)

In [6]:
X_test = df_test['document'].values
y_test = df_test['label'].values

In [7]:
print(len(X_train), np.bincount(y_train))
print(len(X_test), np.bincount(y_test))

150000 [75173 74827]
50000 [24827 25173]


In [8]:
from konlpy.tag import Okt

In [10]:
okt = Okt()
print(X_train[4])
print(okt.morphs(X_train[4]))

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']


In [11]:
# TfidfVectorizer는 기본적으로 공백을 기준으로 토큰은 구분
# tokenizer 매개변수에 토큰화를 위한 사용자 정의 함수를 전달할 수 있음
# okt.morphs 메서드를 전달하면 형태소 분석을 통해 토큰화 수행할 수 있음
# tokenizer 매개변수를 사용할 때 패턴을 token_patter=None으로 지정하여 token_pattern 매개변수가 사용되지 않는다는 경고 메시지 나오지 않게
# TifdfVectorizer ngram_range=(1, 2)로 설정하여 유니그램과 바이그램을 사용하고 min_df=3으로 지정하여 3회 미만으로 등장하는 토큰은 무시.
# max_df=0.9로 두어 가장 많이 등장하는 상위 10% 토큰도 무시
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 2),
                        min_df=3,
                        max_df=0.9,
                        tokenizer=okt.morphs,
                        token_pattern=None)
tfidf.fit(X_train)
X_train_okt = tfidf.transform(X_train)
X_test_okt = tfidf.transform(X_test)                        

In [12]:
# SGDClassifier 클래스를 사용해서 감성 분류 문제 해결
# SGDClassfier의 매개변수는 규제를 위한 alpha 매개변수.
# RandomizedSearchCV 클래스를 사용하기 위해 loguniform 함수로 탐색 범위 지정.
# SGDClassifier의 손실 함수로 로지스틱 손실('log')을 사용하지만 다른 손실 함수를 매개변수 탐색에 포함할 수 있음.
# 총 반복 회수(n_iter)는 50회로 지정
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform
sgd = SGDClassifier(loss='log', random_state=1)
param_dist = {'alpha': loguniform(0.0001, 100.0)}
rsv_okt = RandomizedSearchCV(estimator=sgd,
                            param_distributions=param_dist,
                            n_iter=50,
                            random_state=1,
                            verbose=1,
                            n_jobs=4)
rsv_okt.fit(X_train_okt, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50, n_jobs=4,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f494647ac70>},
                   random_state=1, verbose=1)

In [13]:
# 하이퍼파라미터 탐색으로 찾은 최상의 점수와 매개변수 값
print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

0.8251533333333334
{'alpha': 0.0001001581395585897}


In [14]:
# 테스트 데이터셋 X_test_okt에서 점수 확인
rsv_okt.score(X_test_okt, y_test)

0.8189

In [15]:
# soynlp 세 개의 토큰화 클래스 제공 LTokenizer, MaxScoreTokenizer, RegexTokenizer
# https://github.com/lovit/soynlp
# 띄어쓰기가 잘 되어 있다면 LTokenizer
from soynlp.tokenizer import LTokenizer
lto = LTokenizer()      # 공백으로만 토큰화 수행
print(lto.tokenize(X_train[4]))

['사이몬페그의', '익살스런', '연기가', '돋보였던', '영화!스파이더맨에서', '늙어보이기만', '했던', '커스틴', '던스트가', '너무나도', '이뻐보였다']


In [16]:
from soynlp.word import WordExtractor
word_ext = WordExtractor()
word_ext.train(X_train)
scores = word_ext.word_scores()
# 반환된 scores 객체는 단어마다 결합 점수(cohesion score)와 브랭칭 엔트로피(branching entropy)를 가진 딕셔너리

training was done. used memory 1.308 Gbse memory 1.261 Gb
all cohesion probabilities was computed. # words = 85683
all branching entropies was computed # words = 101540
all accessor variety was computed # words = 101540


In [19]:
import math
score_dict = {key: scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy) for key in scores}
lto = LTokenizer(scores=score_dict)
print(lto.tokenize(X_train[4]))

['사이', '몬페그의', '익살스', '런', '연기', '가', '돋보', '였던', '영화', '!스파이더맨에서', '늙어', '보이기만', '했던', '커스틴', '던스트가', '너무', '나도', '이뻐', '보였다']


In [22]:
tfidf = TfidfVectorizer(ngram_range=(1, 2),
                        min_df=3,
                        max_df=.9,
                        tokenizer=lto.tokenize,
                        token_pattern=None)
tfidf.fit(X_train)
X_train_soy = tfidf.transform(X_train)
X_test_soy = tfidf.transform(X_test)
rsv_soy = RandomizedSearchCV(estimator=sgd,
                            param_distributions=param_dist,
                            n_iter=50,
                            random_state=1,
                            verbose=1,
                            n_jobs=4)
rsv_soy.fit(X_train_soy, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50, n_jobs=4,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f494647ac70>},
                   random_state=1, verbose=1)

In [23]:
print(rsv_soy.best_score_)
print(rsv_soy.best_params_)

0.8141066666666665
{'alpha': 0.0001001581395585897}


In [24]:
rsv_soy.score(X_test_soy, y_test)

0.8085